In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout

In [2]:
df = pd.read_csv("C:\\Users\\Vijay\\Desktop\\dataset.csv")
df

,Class,nring,nrot,ndonr,naccr,dis1,dis2,dis3,0c,1c,2c,3cP,3cC,4cPC,3cCH,4cCH,5cCH,6cCH,0cv,1cv,2cv,3cvP,3cvC,4cvPC,3cvCH,4cvCH,5cvCH,6cvCH,nhyd,nhal,nhet,nhev,ncof,ncocl,ncobr,ncoi,ncarb,nphos,nsulph,noxy,...,S(68),S(69),S(70),S(71),S(72),S(73),S(74),S(75),S(76),S(77),S(78),S(79),S(80),Wmol,Vmc,pi,ea,eb,q+,q-,m,h,SN,IP,A,mcp,cen,w,AS,VS,MS,PSA,Sapc,Sanc,Sapcw,Sancw,Svpc,Svnc,Svpcw,Svncw
0,1,5,5,1,8,14.0728,19.9971,29.7056,23.2753,15.9603,14.5570,12.8918,2.3799,6.0177,0.0000,0.0000,0.2141,0.1481,18.7990,10.6348,8.2686,6.5920,1.0988,2.6485,0.0000,0.0000,0.0776,0.0694,25,0,9,33,0,0,0,0,24,0,0,8,...,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,455.4634,2.0864,0.1759,0.1217,0.1287,0.2321,0.3862,6.7053,4.0776,0.2452,8.4523,0.2971,-4.3747,4.3747,39.0187,682.2858,440.3601,388.7751,93.7600,483.5033,198.7825,50.4138,43.0631,261.6603,178.6998,29.0866,32.9824
1,1,7,8,5,17,18.3987,29.1307,41.9456,66.9766,42.1016,40.5712,34.3946,8.2319,20.6087,0.0000,0.0000,0.2357,0.1481,54.1343,30.5275,25.5439,17.5232,4.4246,8.4235,0.0000,0.0000,0.1826,0.0479,86,0,28,90,0,0,0,0,62,0,0,16,...,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,1255.4322,5.0382,0.1912,0.1072,0.1326,0.2779,0.4471,11.8322,3.5457,0.2820,8.8382,1.7469,-5.2925,5.2925,49.6589,1371.0026,1221.4604,1078.3437,276.8944,1087.3566,283.6460,109.0626,76.9949,754.1281,467.3322,93.6200,112.7599
2,1,5,5,6,11,15.9217,21.4360,31.9111,28.3860,18.4278,17.6913,16.2261,2.8918,8.2157,0.0000,0.0000,0.0000,0.2458,21.2101,12.4118,10.1599,7.8053,1.2748,2.9059,0.0000,0.0000,0.0000,0.1235,29,0,12,39,0,0,0,0,27,0,0,11,...,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,543.5262,2.2310,0.1854,0.1114,0.1313,0.2898,0.3852,2.8958,3.6973,0.2705,8.7147,1.3201,-5.0174,5.0174,46.5381,740.9025,501.9654,431.6854,181.3898,487.8483,253.0542,68.0452,61.5911,266.2773,235.6881,37.6381,49.6918
3,1,4,3,2,5,13.0372,16.8927,24.0154,18.6899,12.3692,11.7933,10.9531,1.3724,3.6814,0.0000,0.0000,0.0833,0.1463,15.3175,9.8899,9.1442,8.2669,0.8264,2.4454,0.0000,0.0000,0.0833,0.1170,28,0,5,26,0,0,0,0,21,0,0,5,...,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,360.4492,1.5117,0.1708,0.1216,0.1483,0.2161,0.3347,1.7924,5.0568,0.1978,10.4141,0.3005,-5.3573,5.3573,72.5659,522.2911,370.6028,293.8360,78.8970,368.0259,154.2652,40.9066,41.4372,221.7073,148.8955,25.5404,30.2031
4,1,3,11,0,3,15.6200,21.8095,32.4662,22.3801,14.9939,12.7462,11.0067,1.7513,4.6964,0.0000,0.0000,0.0786,0.1389,22.6567,13.0462,9.7978,7.0642,1.3412,2.7466,0.0000,0.0000,0.0255,0.0486,29,2,6,31,0,0,0,2,25,0,0,3,...,0,0,0,0,0,0,0,0,0.0,0,0,0,4.5377,645.3180,2.3260,0.1560,0.1175,0.1383,0.1430,0.2618,3.5106,4.3512,0.2298,9.4164,0.7141,-5.0652,5.0652,55.8187,745.5410,484.8534,423.9168,37.9313,577.2155,168.3255,59.2792,25.4594,291.2521,193.6013,31.7277,27.7454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,-1,2,9,2,5,12.7882,19.4653,24.5601,19.2672,12.4003,10.8161,8.7753,1.7138,4.3279,0.0000,0.0000,0.0786,0.0833,16.0710,9.8783,7.4288,5.5977,0.8378,1.9460,0.0000,0.0000,0.0609,0.0833,32,0,7,26,0,0,0,0,19,0,0,5,...,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,368.4722,1.9899,0.1314,0.1339,0.1398,0.2524,0.3744,5.0675,5.2484,0.1905,9.5663,-0.9306,-4.3178,4.3178,48.9249,601.7028,421.1943,339.0629,87.8772,486.6220,115.0808,49.1108,34.4332,267.9176,153.2767,30.8568,35.3643
750,-1,3,7,0,3,12.9131,17.1766,25.7497,18.7338,12.3750,11.7762,8.8997,1.4704,2.6459,0.1667,0.0000,0.0000,0.1854,16.1278,9.1460,7.9592,5.0897,0.8528,1.2666,0.1667,0.0000,0.0000,0.0599,20,2,5,26,0,2,0,0,21,0,0,3,...,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,391.2932,2.1079,0.1450,0.1241,0.1400,0.1576,0.3599,2.3132,4.7619,0.2100,9.5787,0.0549,-4.8168,4.8168,55.2417,621.8020,401.5729,363.4048,30.0842,479.8389,141.9631,45.6243,23.1197,238.2958,163.2771,22.5731,25.2704
751,-1,2,2,2,3,12.1659,13.8090,24.6997,11.3805,7.754

In [3]:
target_columns = ['ndonr', 'naccr', 'nrot', 'Wmol', 'PSA', 'nring', 'nhet', 'Vmc']
X = df.drop(columns=target_columns)
y = df[target_columns]

In [4]:
X

,Class,dis1,dis2,dis3,0c,1c,2c,3cP,3cC,4cPC,3cCH,4cCH,5cCH,6cCH,0cv,1cv,2cv,3cvP,3cvC,4cvPC,3cvCH,4cvCH,5cvCH,6cvCH,nhyd,nhal,nhev,ncof,ncocl,ncobr,ncoi,ncarb,nphos,nsulph,noxy,nnitro,1k,2k,3k,1ka,...,S(65),S(66),S(67),S(68),S(69),S(70),S(71),S(72),S(73),S(74),S(75),S(76),S(77),S(78),S(79),S(80),pi,ea,eb,q+,q-,m,h,SN,IP,A,mcp,cen,w,AS,VS,MS,Sapc,Sanc,Sapcw,Sancw,Svpc,Svnc,Svpcw,Svncw
0,1,14.0728,19.9971,29.7056,23.2753,15.9603,14.5570,12.8918,2.3799,6.0177,0.0000,0.0000,0.2141,0.1481,18.7990,10.6348,8.2686,6.5920,1.0988,2.6485,0.0000,0.0000,0.0776,0.0694,25,0,33,0,0,0,0,24,0,0,8,1,24.6837,10.1660,4.3896,22.1480,...,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,0.1759,0.1217,0.1287,0.2321,0.3862,6.7053,4.0776,0.2452,8.4523,0.2971,-4.3747,4.3747,39.0187,682.2858,440.3601,388.7751,483.5033,198.7825,50.4138,43.0631,261.6603,178.6998,29.0866,32.9824
1,1,18.3987,29.1307,41.9456,66.9766,42.1016,40.5712,34.3946,8.2319,20.6087,0.0000,0.0000,0.2357,0.1481,54.1343,30.5275,25.5439,17.5232,4.4246,8.4235,0.0000,0.0000,0.1826,0.0479,86,0,90,0,0,0,0,62,0,0,16,12,77.3535,33.2377,17.3601,69.2373,...,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,0.1912,0.1072,0.1326,0.2779,0.4471,11.8322,3.5457,0.2820,8.8382,1.7469,-5.2925,5.2925,49.6589,1371.0026,1221.4604,1078.3437,1087.3566,283.6460,109.0626,76.9949,754.1281,467.3322,93.6200,112.7599
2,1,15.9217,21.4360,31.9111,28.3860,18.4278,17.6913,16.2261,2.8918,8.2157,0.0000,0.0000,0.0000,0.2458,21.2101,12.4118,10.1599,7.8053,1.2748,2.9059,0.0000,0.0000,0.0000,0.1235,29,0,39,0,0,0,0,27,0,0,11,1,30.4575,11.5888,5.1279,27.6491,...,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,0.1854,0.1114,0.1313,0.2898,0.3852,2.8958,3.6973,0.2705,8.7147,1.3201,-5.0174,5.0174,46.5381,740.9025,501.9654,431.6854,487.8483,253.0542,68.0452,61.5911,266.2773,235.6881,37.6381,49.6918
3,1,13.0372,16.8927,24.0154,18.6899,12.3692,11.7933,10.9531,1.3724,3.6814,0.0000,0.0000,0.0833,0.1463,15.3175,9.8899,9.1442,8.2669,0.8264,2.4454,0.0000,0.0000,0.0833,0.1170,28,0,26,0,0,0,0,21,0,0,5,0,19.3222,6.8053,2.7037,18.0529,...,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,0.1708,0.1216,0.1483,0.2161,0.3347,1.7924,5.0568,0.1978,10.4141,0.3005,-5.3573,5.3573,72.5659,522.2911,370.6028,293.8360,368.0259,154.2652,40.9066,41.4372,221.7073,148.8955,25.5404,30.2031
4,1,15.6200,21.8095,32.4662,22.3801,14.9939,12.7462,11.0067,1.7513,4.6964,0.0000,0.0000,0.0786,0.1389,22.6567,13.0462,9.7978,7.0642,1.3412,2.7466,0.0000,0.0000,0.0255,0.0486,29,2,31,0,0,0,2,25,0,0,3,1,25.6198,12.4593,6.3209,24.8250,...,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,4.5377,0.1560,0.1175,0.1383,0.1430,0.2618,3.5106,4.3512,0.2298,9.4164,0.7141,-5.0652,5.0652,55.8187,745.5410,484.8534,423.9168,577.2155,168.3255,59.2792,25.4594,291.2521,193.6013,31.7277,27.7454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,-1,12.7882,19.4653,24.5601,19.2672,12.4003,10.8161,8.7753,1.7138,4.3279,0.0000,0.0000,0.0786,0.0833,16.0710,9.8783,7.4288,5.5977,0.8378,1.9460,0.0000,0.0000,0.0609,0.0833,32,0,26,0,0,0,0,19,0,0,5,2,22.2908,10.5186,5.5177,20.9953,...,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,0.1314,0.1339,0.1398,0.2524,0.3744,5.0675,5.2484,0.1905,9.5663,-0.9306,-4.3178,4.3178,48.9249,601.7028,421.1943,339.0629,486.6220,115.0808,49.1108,34.4332,267.9176,153.2767,30.8568,35.3643
750,-1,12.9131,17.1766,25.7497,18.7338,12.3750,11.7762,8.8997,1.4704,2.6459,0.1667,0.0000,0.0000,0.1854,16.1278,9.1460,7.9592,5.0897,0.8528,1.2666,0.1667,0.0000,0.0000,0.0599,20,2,26,0,2,0,0,21,0,0,3,0,20.7270,9.0000,5.0934,19.1210,...,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0000,0.1450,0.1241,0.1400,0.1576,0.3599,2.3132,4.7619,0.2100,9.5787,0.0549,-4.8168,4.8168,55.2417,621.8020,401.5729,363.4048,479.8389,141.9631,45.6243,23.1197,238.2958,163.2771,22.5731,25.2704
751,-1,12.1659,13.8090,24.69

In [5]:
y

,ndonr,naccr,nrot,Wmol,PSA,nring,nhet,Vmc
0,1,8,5,455.4634,93.7600,5,9,2.0864
1,5,17,8,1255.4322,276.8944,7,28,5.0382
2,6,11,5,543.5262,181.3898,5,12,2.2310
3,2,5,3,360.4492,78.8970,4,5,1.5117
4,0,3,11,645.3180,37.9313,3,6,2.3260
...,...,...,...,...,...,...,...,...
749,2,5,9,368.4722,87.8772,2,7,1.9899
750,0,3,7,391.2932,30.0842,3,5,2.1079
751,2,3,2,213.2414,79.4822,2,5,1.1977
752,2,0,3,136.1962,41.8416,1,2,0.8819


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(603, 152)
(151, 152)


Model 1 (XGBRegressor)

In [13]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
reg = XGBRegressor(n_estimators=100, random_state=42)
multi_target_model = MultiOutputRegressor(reg)

multi_target_model.fit(X_train_scaled, y_train)


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
y_pred = multi_target_model.predict(X_test_scaled)

In [103]:
for i, col in enumerate(target_columns):
    test_mae = np.mean(np.abs(y_test.iloc[:, i] - y_pred[:, i]))
    print(f"{col} Test MAE: {test_mae}")

for i in range(5):
    print(f"True values: {y_test.iloc[i].values}, Predicted values: {y_pred[i]}")


ndonr Test MAE: 0.38445620215575005
naccr Test MAE: 0.2671661226091203
nrot Test MAE: 0.8874736830888205
Wmol Test MAE: 9.638206432701264
PSA Test MAE: 9.909368477888297
nring Test MAE: 0.25723833871823154
nhet Test MAE: 0.5345769607468157
Vmc Test MAE: 0.09606140590566674
True values: [  1.       6.       4.     314.2568 104.6289   3.       9.       1.5764], Predicted values: [  1.2642547   5.890565    2.911281  306.48862    99.80409     2.8511329
   8.306692    1.541843 ]
True values: [  4.       4.       3.     197.1902  97.0687   1.       5.       1.0084], Predicted values: [  4.002424    3.889809    3.492592  192.86626    95.14562     0.9955217
   5.207553    1.1416235]
True values: [  0.       6.       3.     352.7707  64.665    3.       7.       1.709 ], Predicted values: [3.5066698e-03 5.8007441e+00 2.6262431e+00 3.4176422e+02 6.3987980e+01
 3.0401821e+00 6.3024554e+00 1.6449195e+00]
True values: [  5.      13.       7.     764.9488 173.9796   8.      13.       3.4171], Predict

In [104]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("R-squared Score:", r2)
rmse = np.sqrt(mse)
print('RMSE: ',rmse)


R-squared Score: 0.9362985491752625
RMSE:  9.24400769583438


Model 2 (Basic NN)

In [105]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
reg = RandomForestRegressor(n_estimators=100, random_state=42)
multi_target_model = MultiOutputRegressor(reg)

multi_target_model.fit(X_train_scaled, y_train)


MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [106]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
y_pred = multi_target_model.predict(X_test_scaled)

In [107]:
for i, col in enumerate(target_columns):
    test_mae = np.mean(np.abs(y_test.iloc[:, i] - y_pred[:, i]))
    print(f"{col} Test MAE: {test_mae}")

for i in range(5):
    print(f"True values: {y_test.iloc[i].values}, Predicted values: {y_pred[i]}")


ndonr Test MAE: 0.3860264900662252
naccr Test MAE: 0.3786754966887418
nrot Test MAE: 0.7601324503311259
Wmol Test MAE: 8.203733602649026
PSA Test MAE: 10.764344768211927
nring Test MAE: 0.26198675496688745
nhet Test MAE: 0.5774834437086092
Vmc Test MAE: 0.09408882119205303
True values: [  1.       6.       4.     314.2568 104.6289   3.       9.       1.5764], Predicted values: [  1.42       5.91       3.65     318.037182 104.063765   3.43
   8.59       1.502956]
True values: [  4.       4.       3.     197.1902  97.0687   1.       5.       1.0084], Predicted values: [  3.82       4.08       3.12     197.397651  89.725589   1.03
   5.39       1.038726]
True values: [  0.       6.       3.     352.7707  64.665    3.       7.       1.709 ], Predicted values: [  0.         5.87       2.65     350.317925  78.551812   3.02
   6.66       1.644937]
True values: [  5.      13.       7.     764.9488 173.9796   8.      13.       3.4171], Predicted values: [  5.89      12.85      12.64     784.518

In [108]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("R-squared Score:", r2)
rmse = np.sqrt(mse)
print('RMSE: ',rmse)


R-squared Score: 0.9509392631278348
RMSE:  7.975697582918256


Model 3 (AdaBoost)

In [109]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.multioutput import MultiOutputRegressor
ada_reg = AdaBoostRegressor(n_estimators=100, random_state=42)
multi_target_model = MultiOutputRegressor(ada_reg)


multi_target_model.fit(X_train_scaled, y_train)


MultiOutputRegressor(estimator=AdaBoostRegressor(n_estimators=100,
                                                 random_state=42))

In [110]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
y_pred = multi_target_model.predict(X_test_scaled)

In [111]:
for i, col in enumerate(target_columns):
    test_mae = np.mean(np.abs(y_test.iloc[:, i] - y_pred[:, i]))
    print(f"{col} Test MAE: {test_mae}")

for i in range(5):
    print(f"True values: {y_test.iloc[i].values}, Predicted values: {y_pred[i]}")


ndonr Test MAE: 0.8062191354619347
naccr Test MAE: 0.7349916851861219
nrot Test MAE: 1.282523691475122
Wmol Test MAE: 19.34282232405081
PSA Test MAE: 17.112330689803006
nring Test MAE: 0.38786452243418446
nhet Test MAE: 0.9692942638714759
Vmc Test MAE: 0.12788017319324244
True values: [  1.       6.       4.     314.2568 104.6289   3.       9.       1.5764], Predicted values: [  1.63888889   6.08421053   3.97916667 309.02644286 106.42401618
   2.98113208   8.70618557   1.50006744]
True values: [  4.       4.       3.     197.1902  97.0687   1.       5.       1.0084], Predicted values: [  2.90163934   4.68131868   3.04580153 216.29354776  93.65687385
   1.19354839   6.51260504   1.089075  ]
True values: [  0.       6.       3.     352.7707  64.665    3.       7.       1.709 ], Predicted values: [  1.27741935   5.55801105   3.55555556 349.60885629  89.05074583
   2.85185185   7.28654971   1.5889265 ]
True values: [  5.      13.       7.     764.9488 173.9796   8.      13.       3.4171], 

In [112]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("R-squared Score:", r2)
rmse = np.sqrt(mse)
print('RMSE: ',rmse)

R-squared Score: 0.9191896980258492
RMSE:  11.602425342829617


Model 4 (K-Nearest Neighbors Regressor)

In [113]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor

knn_reg = KNeighborsRegressor(n_neighbors=5)
multi_target_model = MultiOutputRegressor(knn_reg)

multi_target_model.fit(X_train_scaled, y_train)


MultiOutputRegressor(estimator=KNeighborsRegressor())

In [114]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
y_pred = multi_target_model.predict(X_test_scaled)

In [115]:
for i, col in enumerate(target_columns):
    test_mae = np.mean(np.abs(y_test.iloc[:, i] - y_pred[:, i]))
    print(f"{col} Test MAE: {test_mae}")

for i in range(5):
    print(f"True values: {y_test.iloc[i].values}, Predicted values: {y_pred[i]}")


ndonr Test MAE: 0.6741721854304636
naccr Test MAE: 0.9827814569536423
nrot Test MAE: 1.4847682119205297
Wmol Test MAE: 32.78563245033113
PSA Test MAE: 16.164058807947022
nring Test MAE: 0.5059602649006623
nhet Test MAE: 1.0953642384105962
Vmc Test MAE: 0.1872629139072848
True values: [  1.       6.       4.     314.2568 104.6289   3.       9.       1.5764], Predicted values: [  0.8       5.6       3.2     249.80844  86.9778    2.2       7.4
   1.25242]
True values: [  4.       4.       3.     197.1902  97.0687   1.       5.       1.0084], Predicted values: [  3.2       2.8       2.2     175.99188  73.09718   1.        3.8
   0.9561 ]
True values: [  0.       6.       3.     352.7707  64.665    3.       7.       1.709 ], Predicted values: [2.0000000e-01 4.0000000e+00 3.6000000e+00 3.0958558e+02 4.9123080e+01
 2.8000000e+00 5.2000000e+00 1.4848600e+00]
True values: [  5.      13.       7.     764.9488 173.9796   8.      13.       3.4171], Predicted values: [  4.8      12.4       7.6     

In [116]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("R-squared Score:", r2)
rmse = np.sqrt(mse)
print('RMSE: ',rmse)

R-squared Score: 0.8469822945874337
RMSE:  19.23633608013691


Model 5 (Lasso Regressor)

In [117]:
from sklearn.linear_model import Lasso
from sklearn.multioutput import MultiOutputRegressor

lasso_reg = Lasso(alpha=0.1)
multi_target_model = MultiOutputRegressor(lasso_reg)

multi_target_model.fit(X_train_scaled, y_train)


c:\Users\Vijay\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.429e+03, tolerance: 1.868e+03
  model = cd_fast.enet_coordinate_descent(


MultiOutputRegressor(estimator=Lasso(alpha=0.1))

In [118]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
y_pred = multi_target_model.predict(X_test_scaled)

In [119]:
for i, col in enumerate(target_columns):
    test_mae = np.mean(np.abs(y_test.iloc[:, i] - y_pred[:, i]))
    print(f"{col} Test MAE: {test_mae}")

for i in range(5):
    print(f"True values: {y_test.iloc[i].values}, Predicted values: {y_pred[i]}")


ndonr Test MAE: 0.4952408538182011
naccr Test MAE: 0.31325715562698936
nrot Test MAE: 1.1414734289088673
Wmol Test MAE: 0.8885527415557601
PSA Test MAE: 4.475064786822687
nring Test MAE: 0.3291692021771359
nhet Test MAE: 0.21950611580141852
Vmc Test MAE: 0.13701620466036588
True values: [  1.       6.       4.     314.2568 104.6289   3.       9.       1.5764], Predicted values: [  1.52423415   5.61220198   4.78853779 314.36271821 107.2916015
   3.2990867    8.66691982   1.56551279]
True values: [  4.       4.       3.     197.1902  97.0687   1.       5.       1.0084], Predicted values: [  3.51313582   4.07713001   2.88909445 197.66655228  91.87900445
   1.25265999   5.1645848    1.139485  ]
True values: [  0.       6.       3.     352.7707  64.665    3.       7.       1.709 ], Predicted values: [-6.46764149e-03  5.95340860e+00  2.94091844e+00  3.54273814e+02
  5.84708403e+01  2.84837023e+00  6.87482009e+00  1.56078893e+00]
True values: [  5.      13.       7.     764.9488 173.9796   8.

In [120]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("R-squared Score:", r2)
rmse = np.sqrt(mse)
print('RMSE: ',rmse)

R-squared Score: 0.9475158295900028
RMSE:  2.7539401736571776


Model 6 (Ridge Regressor)

In [121]:
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor

ridge_reg = Ridge(alpha=1.0)
multi_target_model = MultiOutputRegressor(ridge_reg)

multi_target_model.fit(X_train_scaled, y_train)


MultiOutputRegressor(estimator=Ridge())

In [122]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
y_pred = multi_target_model.predict(X_test_scaled)

In [123]:
for i, col in enumerate(target_columns):
    test_mae = np.mean(np.abs(y_test.iloc[:, i] - y_pred[:, i]))
    print(f"{col} Test MAE: {test_mae}")

for i in range(5):
    print(f"True values: {y_test.iloc[i].values}, Predicted values: {y_pred[i]}")


ndonr Test MAE: 0.28265758274609115
naccr Test MAE: 0.1485476098137811
nrot Test MAE: 1.005838396828569
Wmol Test MAE: 0.809126529670887
PSA Test MAE: 3.929199704032443
nring Test MAE: 0.08052036601686104
nhet Test MAE: 0.052210038865926615
Vmc Test MAE: 0.03939816919419309
True values: [  1.       6.       4.     314.2568 104.6289   3.       9.       1.5764], Predicted values: [  1.3077222    5.89625106   4.17637723 314.85912199 108.67156001
   3.05807223   8.95131115   1.54022837]
True values: [  4.       4.       3.     197.1902  97.0687   1.       5.       1.0084], Predicted values: [  3.72608919   3.93780149   2.95642999 197.97746991  92.66903698
   1.09610974   4.94887543   1.00763281]
True values: [  0.       6.       3.     352.7707  64.665    3.       7.       1.709 ], Predicted values: [-1.55342526e-01  5.95077696e+00  3.56007944e+00  3.52558697e+02
  5.97705497e+01  3.02137635e+00  7.12142670e+00  1.69176717e+00]
True values: [  5.      13.       7.     764.9488 173.9796   8

In [124]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("R-squared Score:", r2)
rmse = np.sqrt(mse)
print('RMSE: ',rmse)

R-squared Score: 0.9817615674235023
RMSE:  2.2015806313320203


Model 7 (Gradient Boosting Regressor)

In [125]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor

gbr = GradientBoostingRegressor(n_estimators=100, random_state=42)

multi_target_model = MultiOutputRegressor(gbr)
multi_target_model.fit(X_train_scaled, y_train)


MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=42))

In [126]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
y_pred = multi_target_model.predict(X_test_scaled)

In [127]:
for i, col in enumerate(target_columns):
    test_mae = np.mean(np.abs(y_test.iloc[:, i] - y_pred[:, i]))
    print(f"{col} Test MAE: {test_mae}")

for i in range(5):
    print(f"True values: {y_test.iloc[i].values}, Predicted values: {y_pred[i]}")


ndonr Test MAE: 0.3612419249328421
naccr Test MAE: 0.2888597003818899
nrot Test MAE: 0.7758078383343762
Wmol Test MAE: 7.791835160738132
PSA Test MAE: 7.790758126421266
nring Test MAE: 0.25516249250939865
nhet Test MAE: 0.4463683177332712
Vmc Test MAE: 0.0737780213220409
True values: [  1.       6.       4.     314.2568 104.6289   3.       9.       1.5764], Predicted values: [  1.23619853   5.73824814   3.26103931 309.89416381  99.82058533
   3.14684596   8.73042984   1.45680626]
True values: [  4.       4.       3.     197.1902  97.0687   1.       5.       1.0084], Predicted values: [  3.78676322   4.11293849   3.24142835 197.94421204  93.93051748
   0.93261408   5.14307576   1.012876  ]
True values: [  0.       6.       3.     352.7707  64.665    3.       7.       1.709 ], Predicted values: [1.71251033e-02 5.78069376e+00 3.18567859e+00 3.47282601e+02
 7.15608088e+01 2.89970728e+00 7.14438349e+00 1.56913588e+00]
True values: [  5.      13.       7.     764.9488 173.9796   8.      13. 

In [128]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("R-squared Score:", r2)
rmse = np.sqrt(mse)
print('RMSE: ',rmse)

R-squared Score: 0.9678951677017482
RMSE:  5.814475962455798


Model 8 (Linear Regression)

In [129]:
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

linear_reg = LinearRegression()

multi_target_model = MultiOutputRegressor(linear_reg)
multi_target_model.fit(X_train_scaled, y_train)


MultiOutputRegressor(estimator=LinearRegression())

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
y_pred = multi_target_model.predict(X_test_scaled)

In [131]:
for i, col in enumerate(target_columns):
    test_mae = np.mean(np.abs(y_test.iloc[:, i] - y_pred[:, i]))
    print(f"{col} Test MAE: {test_mae}")

for i in range(5):
    print(f"True values: {y_test.iloc[i].values}, Predicted values: {y_pred[i]}")


ndonr Test MAE: 0.21604928650686617
naccr Test MAE: 0.14470674927148344
nrot Test MAE: 1.0022942331447606
Wmol Test MAE: 1.3756355538541376e-06
PSA Test MAE: 3.0132263189683326
nring Test MAE: 0.034778733649804966
nhet Test MAE: 1.1343391274117162e-14
Vmc Test MAE: 0.04088654516564063
True values: [  1.       6.       4.     314.2568 104.6289   3.       9.       1.5764], Predicted values: [  0.96202601   5.93320271   3.46561606 314.25680106 106.04175442
   3.01423173   9.           1.55649831]
True values: [  4.       4.       3.     197.1902  97.0687   1.       5.       1.0084], Predicted values: [  3.85785712   3.98919275   2.3663041  197.19019763  95.44908094
   0.98991162   5.           0.99904353]
True values: [  0.       6.       3.     352.7707  64.665    3.       7.       1.709 ], Predicted values: [-5.85017870e-02  5.82878874e+00  4.15848594e+00  3.52770703e+02
  5.89842096e+01  3.02539990e+00  7.00000000e+00  1.71782082e+00]
True values: [  5.      13.       7.     764.9488 1

In [132]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("R-squared Score:", r2)
rmse = np.sqrt(mse)
print('RMSE: ',rmse)

R-squared Score: 0.9831768904112457
RMSE:  1.6139422786617585


Model 9 (Decision Tree Regressor)

In [133]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor

dt_reg = DecisionTreeRegressor(random_state=42)
multi_target_model = MultiOutputRegressor(dt_reg)

multi_target_model.fit(X_train_scaled, y_train)


MultiOutputRegressor(estimator=DecisionTreeRegressor(random_state=42))

In [134]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
y_pred = multi_target_model.predict(X_test_scaled)

In [135]:
for i, col in enumerate(target_columns):
    test_mae = np.mean(np.abs(y_test.iloc[:, i] - y_pred[:, i]))
    print(f"{col} Test MAE: {test_mae}")

for i in range(5):
    print(f"True values: {y_test.iloc[i].values}, Predicted values: {y_pred[i]}")


ndonr Test MAE: 0.36423841059602646
naccr Test MAE: 0.39072847682119205
nrot Test MAE: 0.9006622516556292
Wmol Test MAE: 13.1814880794702
PSA Test MAE: 17.74912715231788
nring Test MAE: 0.33112582781456956
nhet Test MAE: 0.8609271523178808
Vmc Test MAE: 0.11638410596026491
True values: [  1.       6.       4.     314.2568 104.6289   3.       9.       1.5764], Predicted values: [  1.       6.       5.     331.2841  83.2454   2.       9.       1.3463]
True values: [  4.       4.       3.     197.1902  97.0687   1.       5.       1.0084], Predicted values: [  4.       5.       4.     182.1698  95.4047   1.       5.       1.0715]
True values: [  0.       6.       3.     352.7707  64.665    3.       7.       1.709 ], Predicted values: [  0.       6.       2.     350.3886  97.8471   3.       7.       1.5809]
True values: [  5.      13.       7.     764.9488 173.9796   8.      13.       3.4171], Predicted values: [  6.      13.      17.     780.9482 148.5378   8.       9.       3.3405]
True v

In [136]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("R-squared Score:", r2)
rmse = np.sqrt(mse)
print('RMSE: ',rmse)

R-squared Score: 0.89115294493335
RMSE:  12.791771789198274
